In [1]:
import cv2
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.feature import graycomatrix, graycoprops
from skimage.feature import canny

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
#for canny edge detection
def canny_edge(image,t1=50,t2=130):
  return cv2.Canny(image,t1,t2)

In [3]:
#for denoising
def denoising(image):
  return cv2.GaussianBlur(image,(5,5),0)

In [4]:
#for contrast enhancement
def clahe_hist(image):
  clahe=cv2.createCLAHE(clipLimit=2)
  return clahe.apply(image)

In [5]:
#for saving the GLCM properties
properties=['contrast','dissimilarity','homogeneity','energy','correlation']
distance=[1,3,5,9]
angles=[0, np.pi/4, np.pi/2, 3*np.pi/4, 5*np.pi/4, 3*np.pi/2, 7*np.pi/4]

In [6]:
#to get the GLCM property values of the given image
def glcm_image(image):
  glcm=graycomatrix(image,distance,angles,256,[0,256])
  #print(glcm)
  return glcm

In [7]:
#for data preprocessing
def image_preprocess(path):
  image_data=[]
  file_names=[]
  features=[]
  columns=[]

  #read the sub folders/classes
  for file_name in os.listdir(path):
    file_path=os.path.join(path,file_name)
    for image_path in os.listdir(file_path):
      img=imread(os.path.join(file_path,image_path))

      #check if the image has the RGB band or not
      if img.ndim == 3:
        gray_img = rgb2gray(img)
        gray_img = (gray_img * 255).astype(np.uint8)
      else:
        gray_img = img

      #pass the image through different functions
      edges=denoising(gray_img)
      edges=clahe_hist(edges)
      edges=canny_edge(edges)

      glcm=glcm_image(edges)

      #prepare the feature list, which has 7*4*5=140 features for each image
      image_features=[]
      for prop in properties:
        values=graycoprops(glcm,prop)
        image_features.extend(values.flatten())

      features.append(image_features)
      file_names.append(file_name)

  #create a column list for the 140 features to be appended inside the dataframe
  for prop in properties:
    for d in distance:
      for a in angles:
          columns.append(f'{prop}_d{d}_a{int(np.degrees(a))}')

  #dataframe creation
  df=pd.DataFrame(features,columns=columns)
  df['Class Label']=file_names
  return df

In [ ]:
train_path=r'/content/drive/MyDrive/Colab Notebooks/Bone Fracture Classification Dataset/train'
test_path=r'/content/drive/MyDrive/Colab Notebooks/Bone Fracture Classification Dataset/test'

In [ ]:
#create dataframes for the training and testing data
df_train=image_preprocess(train_path)
print(df_train.shape)

df_test=image_preprocess(test_path)
print(df_test.shape)

#splitting the dataframe to the features and classes for test and train data
x_train=df_train.iloc[:,:-1]#selects all rows and (columns except the last)
y_train=df_train.iloc[:,-1]#select all rows and the last colum

x_test=df_test.iloc[:,:-1]
y_test=df_test.iloc[:,-1]

(8865, 141)
(600, 141)


In [ ]:
#creating the SVM model
svm=SVC(kernel='rbf',C=1)
svm.fit(x_train,y_train)

SVC(C=1)

In [ ]:
#running the predictions with the SVM model
svm_prediction=svm.predict(x_test)

accuracy=accuracy_score(y_test,svm_prediction)
print("Accuracy:",accuracy)

Accuracy: 0.705


In [ ]:
#precision matrices
accuracy=accuracy_score(y_test,svm_prediction)
precision=precision_score(y_test,svm_prediction,average='weighted')
recall=recall_score(y_test,svm_prediction,average='weighted')
f1=f1_score(y_test,svm_prediction,average='weighted')

class_report=classification_report(y_test,svm_prediction)

print("Accuracy=",accuracy)
print("Precision=",precision)
print("Recall=",recall)
print("F1 score=",f1)
print("Classification Report:")
print(class_report)

Accuracy= 0.705
Precision= 0.7085421994884911
Recall= 0.705
F1 score= 0.7063313990973565
Classification Report:
               precision    recall  f1-score   support

    fractured       0.77      0.73      0.75       360
not fractured       0.62      0.66      0.64       240

     accuracy                           0.70       600
    macro avg       0.69      0.70      0.70       600
 weighted avg       0.71      0.70      0.71       600



In [ ]:
import joblib
joblib.dump(svm,'/content/drive/MyDrive/Colab Notebooks/svm_model.pkl')

['/content/drive/MyDrive/Colab Notebooks/svm_model.pkl']

In [ ]:
decision_tree=DecisionTreeClassifier(random_state=42)
decision_tree.fit(x_train,y_train)

DecisionTreeClassifier(random_state=42)

In [ ]:
dt_prediction=decision_tree.predict(x_test)

accuracy=accuracy_score(y_test,dt_prediction)
print("Accuracy:",accuracy)

Accuracy: 0.65


In [ ]:
#precision matrices
accuracy=accuracy_score(y_test,dt_prediction)
precision=precision_score(y_test,dt_prediction,average='weighted')
recall=recall_score(y_test,dt_prediction,average='weighted')
f1=f1_score(y_test,dt_prediction,average='weighted')

class_report=classification_report(y_test,dt_prediction)

print("Accuracy=",accuracy)
print("Precision=",precision)
print("Recall=",recall)
print("F1 score=",f1)
print("Classification Report:")
print(class_report)

Accuracy= 0.65
Precision= 0.6515088879702357
Recall= 0.65
F1 score= 0.6506899055918663
Classification Report:
               precision    recall  f1-score   support

    fractured       0.71      0.70      0.71       360
not fractured       0.56      0.57      0.57       240

     accuracy                           0.65       600
    macro avg       0.64      0.64      0.64       600
 weighted avg       0.65      0.65      0.65       600



In [ ]:
#building the KNN classifier with the training data

KNN_Classifier=KNeighborsClassifier(n_neighbors=5)#preferred value is 5 nearest neighbors
KNN_Classifier.fit(x_train,y_train)

#convert x_test to np.array
x_test_np=x_test.values

knn_training_pred=KNN_Classifier.predict(x_test_np)

accuracy=accuracy_score(y_test,knn_training_pred)
print("Accuracy:",accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


Accuracy: 0.625


In [ ]:
#precision matrices
accuracy=accuracy_score(y_test,knn_training_pred)
precision=precision_score(y_test,knn_training_pred,average='weighted')
recall=recall_score(y_test,knn_training_pred,average='weighted')
f1=f1_score(y_test,knn_training_pred,average='weighted')

class_report=classification_report(y_test,knn_training_pred)

print("Accuracy=",accuracy)
print("Precision=",precision)
print("Recall=",recall)
print("F1 score=",f1)
print("Classification Report:")
print(class_report)

Accuracy= 0.625
Precision= 0.6303513400287595
Recall= 0.625
F1 score= 0.6270572528795108
Classification Report:
               precision    recall  f1-score   support

    fractured       0.70      0.66      0.68       360
not fractured       0.53      0.57      0.55       240

     accuracy                           0.62       600
    macro avg       0.61      0.62      0.61       600
 weighted avg       0.63      0.62      0.63       600



In [ ]:
#training the Random forest classifier with the given training dataset
random_forest_training = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_training.fit(x_train, y_train)

#Evaluate the Model
rf_pred = random_forest_training.predict(x_test)
accuracy = accuracy_score(y_test, rf_pred)
print("Accuracy:", accuracy)

Accuracy: 0.685


In [ ]:
#precision matrices
accuracy=accuracy_score(y_test,rf_pred)
precision=precision_score(y_test,rf_pred,average='weighted')
recall=recall_score(y_test,rf_pred,average='weighted')
f1=f1_score(y_test,rf_pred,average='weighted')

class_report=classification_report(y_test,rf_pred)

print("Accuracy=",accuracy)
print("Precision=",precision)
print("Recall=",recall)
print("F1 score=",f1)
print("Classification Report:")
print(class_report)

Accuracy= 0.685
Precision= 0.6910087892498111
Recall= 0.685
F1 score= 0.6870045152328497
Classification Report:
               precision    recall  f1-score   support

    fractured       0.75      0.71      0.73       360
not fractured       0.60      0.65      0.62       240

     accuracy                           0.69       600
    macro avg       0.68      0.68      0.68       600
 weighted avg       0.69      0.69      0.69       600



In [ ]:
#naive bayes classifier
naive_bayes=GaussianNB()
naive_bayes.fit(x_train,y_train)

nb_predict=naive_bayes.predict(x_test)
accuracy=accuracy_score(y_test,nb_predict)
print("Accuracy:",accuracy)

Accuracy: 0.5866666666666667


In [ ]:
#precision matrices
accuracy=accuracy_score(y_test,nb_predict)
precision=precision_score(y_test,nb_predict,average='weighted')
recall=recall_score(y_test,nb_predict,average='weighted')
f1=f1_score(y_test,nb_predict,average='weighted')

class_report=classification_report(y_test,nb_predict)

print("Accuracy=",accuracy)
print("Precision=",precision)
print("Recall=",recall)
print("F1 score=",f1)
print("Classification Report:")
print(class_report)

Accuracy= 0.5866666666666667
Precision= 0.5860949904823808
Recall= 0.5866666666666667
F1 score= 0.5863744364213772
Classification Report:
               precision    recall  f1-score   support

    fractured       0.65      0.66      0.66       360
not fractured       0.48      0.48      0.48       240

     accuracy                           0.59       600
    macro avg       0.57      0.57      0.57       600
 weighted avg       0.59      0.59      0.59       600

